In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# ! pip install gym

In [3]:
from gym.spaces import Discrete
from gym.spaces import Tuple
from gym.spaces.box import Box

In [4]:
throttle_brake = Discrete(3)  # -1 brake, 0 keep, 1 throttle
steering = Discrete(3)
action_space = Tuple([throttle_brake,steering])

N = 4 # FIXME not hard code
F = 6 # FIXME not hard code
state_space = Box(low=-np.inf, high=np.inf, shape=(N,F), dtype=np.float32)

In [6]:
from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Reshape, Flatten, Masking, LSTM, Concatenate
from tensorflow.keras.models import Model



In [14]:
CAV_state_input = Input(name="cav_input",shape=(6,))
CAV_action_input = Input(name="cav_action",shape=(3,)) #throttle, brake, steering
input_concate = Concatenate(axis=-1)([CAV_state_input,CAV_action_input])
x = Dense(32,activation='relu',name="CAV_dynamics_1")(input_concate)
x = Dense(16,activation='relu',name="CAV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
CAV_dynamics = Model(inputs=[CAV_state_input,CAV_action_input], outputs=out)
CAV_dynamics.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cav_input (InputLayer)          [(None, 6)]          0                                            
__________________________________________________________________________________________________
cav_action (InputLayer)         [(None, 3)]          0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 9)            0           cav_input[0][0]                  
                                                                 cav_action[0][0]                 
__________________________________________________________________________________________________
CAV_dynamics_1 (Dense)          (None, 32)           320         concatenate_7[0][0]        

In [17]:
HDV_state_input = Input(name="hdv_input",shape=(6,))
x = Dense(32,activation='relu',name="HDV_dynamics_1")(HDV_state_input)
x = Dense(16,activation='relu',name="HDV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
HDV_dynamics = Model(inputs=HDV_state_input, outputs=out)
HDV_dynamics.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hdv_input (InputLayer)       [(None, 6)]               0         
_________________________________________________________________
HDV_dynamics_1 (Dense)       (None, 32)                224       
_________________________________________________________________
HDV_dynamics_2 (Dense)       (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 102       
Total params: 854
Trainable params: 854
Non-trainable params: 0
_________________________________________________________________


# Reverse the control commands

In [7]:
import glob
import pickle

In [10]:
!pwd

/Users/jiqiandong/Desktop/OneDrive - purdue.edu/work2/carla_rl


In [12]:
with open("./control_details/LHDV.p", 'rb') as f:
    commands = pickle.load(f)

In [16]:
commands_reversed = []
for command in commands:
    c = command.copy()
    c['steer'] = -command['steer']
    commands_reversed.append(c)

In [17]:
commands_reversed[50]

{'throttle': 0.0,
 'steer': -0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [18]:
commands[50]

{'throttle': 0.0,
 'steer': 0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [19]:
import random

In [31]:
random.choice([0,1])

0

In [32]:
with open("./control_details/LHDV_right.p", 'wb') as f:
    pickle.dump(commands_reversed,f)

In [1]:
import numpy as np

In [2]:
a = np.array([[1,2,3],[4,5,6]])
b = a*2

In [3]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [4]:
b

array([[ 2,  4,  6],
       [ 8, 10, 12]])

In [7]:
np.mean([a,b],axis=0)

array([[1.5, 3. , 4.5],
       [6. , 7.5, 9. ]])

In [1]:
a = {'1':1,'b':2}

In [3]:
for key,val in a.items():
    print(key,val)

1 1
b 2


In [6]:
for item in a:
    print(a[item])

1
2


In [11]:
a = [[[1,2,3],[4,5,6]],[[1,2,3],[4,5,6]]]

In [12]:
a

[[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]]

In [14]:
import numpy as np
np.array(a)[0]

array([[1, 2, 3],
       [4, 5, 6]])

## Test dataset

In [16]:
import pickle
with open('./experience_data/data_pickle.pickle','rb') as f:
    d = pickle.load(f)

In [22]:
for data in d.random_iterator(3):
    print(data)

 4.79995584e+00,  1.86189003e+01,
          2.68388450e-01, -4.01472002e-01, -6.10810798e-03],
        [ 1.16921700e+02,  4.81120348e+00,  1.86029415e+01,
          2.68146038e-01, -3.67044926e-01, -5.57536958e-03],
        [ 1.17730057e+02,  4.82285547e+00,  1.85864124e+01,
          2.67895073e-01, -3.64548266e-01, -5.53503074e-03]],

       [[ 1.14207527e+02,  4.78441715e+00,  1.86573715e+01,
          2.75174826e-01, -3.86457145e-01, -6.03211205e-03],
        [ 1.15003845e+02,  4.79616117e+00,  1.86410770e+01,
          2.74920791e-01, -3.83669883e-01, -5.98150957e-03],
        [ 1.15795067e+02,  4.80783033e+00,  1.86248894e+01,
          2.74669021e-01, -3.64480019e-01, -5.66883059e-03],
        [ 1.16621758e+02,  4.82002163e+00,  1.86079788e+01,
          2.74406105e-01, -3.68802011e-01, -5.73393330e-03],
        [ 1.17474442e+02,  4.83259583e+00,  1.85905418e+01,
          2.74135441e-01, -4.18411911e-01, -6.49477867e-03]],

       [[ 1.14286217e+02,  4.77411413e+00,  1.86575356

In [26]:
dir()

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [38]:
next(d.random_iterator(3))[2]

defaultdict(list,
            {'BHDV': array([[[ 7.53269730e+01,  7.72915411e+00,  4.81271648e+00,
                      -3.58154997e-02, -7.69777149e-02,  7.55276158e-02],
                     [ 7.55631866e+01,  7.72678471e+00,  4.80861425e+00,
                      -3.20573188e-02, -9.72598195e-02,  8.91027525e-02],
                     [ 7.57659225e+01,  7.72491980e+00,  4.80509186e+00,
                      -2.93745846e-02, -8.10073465e-02,  6.16969727e-02],
                     [ 7.59747620e+01,  7.72350979e+00,  4.80137205e+00,
                      -1.33879073e-02, -8.27390254e-02,  3.55588943e-01],
                     [ 7.61905212e+01,  7.72244692e+00,  4.79758644e+00,
                      -5.59463771e-03, -8.52956921e-02,  1.75594494e-01]],
             
                    [[ 7.85644073e+01,  7.72363615e+00,  5.79485464e+00,
                       3.90797965e-02,  3.75913453e+00,  1.10315859e-01],
                     [ 7.88163834e+01,  7.72493267e+00,  5.91791010e+00,
    